## Cómo construir un Chatbot avanzado con memoria de sesión usando LangChain
* Podrá tener diferentes memorias para diferentes sesiones de usuario
* Podrá recordar una cantidad limitada de mensajes: Memoria limitada

In [1]:
import warnings
import os

from langchain._api import LangChainDeprecationWarning
from dotenv import load_dotenv, find_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory, FileChatMessageHistory
warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

_ = load_dotenv(find_dotenv())
openai_api_key = os.environ['OPENAI_API_KEY']

# Creamos el modelo a utilizar
chatModel = ChatOpenAI(model="gpt-4o-mini")

# Creamos nuestro formateador de salida
output_parser = StrOutputParser()

In [2]:
messagesToTheChatbot = [
    HumanMessage(content="Mi color favorito es el azul."),
]
chatModel.invoke(messagesToTheChatbot)

AIMessage(content='¡El azul es un color hermoso! Se asocia a menudo con la tranquilidad, la paz y la serenidad. Además, hay muchas tonalidades de azul, desde el claro hasta el oscuro. ¿Tienes alguna tonalidad favorita o un significado especial que le atribuyas al color azul?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 14, 'total_tokens': 74, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9b78b61c52', 'finish_reason': 'stop', 'logprobs': None}, id='run-ef00fb5a-a76e-4e07-b1ef-f7b8b6330973-0', usage_metadata={'input_tokens': 14, 'output_tokens': 60, 'total_tokens': 74, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Vamos a agregar memoria al Chatbot
* Usaremos el paquete `ChatMessageHistory`
* Guardaremos la memoria en un diccionario de Python llamado `chatbotMemory`
* Definiremos la función `get_session_history` para crear una `session_id` para cada conversación
* Usaremos el Runnable integrado `RunnableWithMessagehistory`

In [3]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {} # Almacenaremos las sesiones IDs y sus respectivos historial de chat

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]

chatbot_with_message_history = RunnableWithMessageHistory(
    chatModel,
    get_session_history
)

In [4]:
# Configuramos una primera sesión
session1 = {"configurable" : { "session_id" : "001"}}

In [21]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi color favorito es el rojo")],
    config=session1,
)

In [20]:
responseFromChatbot.content

'¡Sí, tu color favorito es el rojo! Es un color lleno de energía y pasión. Si tienes algo más que quieras compartir sobre tu amor por el rojo, ¡estaré encantado de escucharlo!'

In [7]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi color favorito?")],
    config=session1,
)

responseFromChatbot.content

'Tu color favorito es el rojo. ¡Es un color muy llamativo y lleno de carácter! ¿Te gusta usarlo en tu ropa o en la decoración?'

### Ahora vamos a cambiar la `session_id` y ver que sucede

Vamos a crear una nueva memoria para un chat pero para otro usuario

In [8]:
session2 = {"configurable" : { "session_id" : "002"}}

In [9]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cuál es mi color favorito?")],
    config=session2,
)

responseFromChatbot.content

'No tengo acceso a información personal sobre ti, así que no puedo saber cuál es tu color favorito. Pero si quieres, puedes compartírmelo o describir tus preferencias y puedo ayudarte a encontrar más información sobre ese color.'

##### Vamos a validar que el Chatbot aún recuerde la memoria de la sesión 001

In [11]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Recuerdas cual es mi color favorito?")],
    config=session1,
)

responseFromChatbot.content

'Sí, tu color favorito es el rojo. Si quieres hablar más sobre eso o compartir por qué te gusta, ¡estoy aquí para escucharte!'

##### Ahora vamos a definir una función para limitar el número de mensajes guardados en memoria y agregar esto a nuestra cadena con `.assign`

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un asistente útil. Responde todas las preguntas lo mejor que puedas.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt
    | chatModel
)

In [14]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

##### Vamos a agregar 2 mensajes adicionales a la sesión 1

In [15]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi equipo de fútbol favorito es el FC Barcelona")],
    config=session1,
)

responseFromChatbot.content

'¡El FC Barcelona es un gran equipo con una rica historia y muchos éxitos! Tienen una gran cantidad de aficionados en todo el mundo y un estilo de juego muy característico. ¿Tienes algún jugador favorito del Barça o algún partido que recuerdes con especial cariño?'

In [16]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi ciudad favorita podría ser Texas")],
    config=session1,
)

responseFromChatbot.content

'Texas es un estado con una gran diversidad cultural y paisajes impresionantes. Hay muchas ciudades interesantes en Texas, como Houston, Austin, Dallas y San Antonio, cada una con su propia personalidad y atracciones. ¿Hay alguna ciudad en particular de Texas que te guste más o que hayas visitado?'

##### La memoria del Chatbot ahora tiene 4 mensajes. Vamos a validar si recuerda lo anterior

In [17]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="¿Recuerdas cual es mi color favorito?")],
    },
    config=session1,
)

responseFromChatbot.content

'No tengo acceso a información previa sobre ti, así que no sé cuál es tu color favorito. Si me lo dices, estaré encantado de recordarlo durante nuestra conversación. ¿Cuál es tu color favorito?'

##### Sin embargo, el primer Chatbo sin límite de memoria si recuerda nuestro historial de conversación

In [ ]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="¿Cual es mi color favorito")],
    config=session1,
)

responseFromChatbot.content

"Your favorite color is red. If you'd like to share more about why you like it or anything else related to it, feel free to let me know!"